<a href="https://colab.research.google.com/github/silkthyme/treehacks2020/blob/master/chained_widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install ipyleaflet
import datetime
import dateutil.parser
import ipywidgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import ee
import ipyleaflet

     |████████████████████████████████| 2.8MB 3.0MB/s 


In [7]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/wgHnqWmWzKEPEahCV9z0NFIVcpBRyv--Rd1MgZfc3dEH37qKwKdfONA

Successfully saved authorization token.


In [49]:
images = ee.ImageCollection('MODIS/006/MOD10A1').filter(ee.Filter.date('2018-01-01', '2018-12-31'))
region = ee.Feature(ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(ee.Filter.eq('us_l3name', 'Sierra Nevada')).first())
image = images.first()
image.bandNames().getInfo()
images.size().getInfo()

364

In [0]:
#get the monday of every week in a month
def monthly(start_date, end_date):
  weeks = []
  start_date = dt.strptime(start_date, '%Y-%m-%d')
  end_date = dt.strptime(end_date, '%Y-%m-%d')
  while start_date <= end_date:
    ref_date = start_date
    monday = ref_date - timedelta(days = ref_date.weekday())
    weeks.append(ref_date)
    start_date += timedelta(days = 33)
  return weeks

#modified from user tda on gis.stackexchange
def extract_time_series(collection, geometry, start, end):
    # Get list of images which correspond with the above
    weeks = get_mondays(start,end)
    store = []
    date_store = []
    images = [item.get('id') for item in collection.getInfo().get('features')]
    for image in images:
      image = ee.Image(image)
      # Obtain date from timestamp in metadata
      date_store.append(np.datetime64(dt.fromtimestamp(image.get("system:time_start").getInfo() / 1000.)))
      data = temp_collection.reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=30, maxPixels=1e9)      
      store.append(data.getInfo()['NDSI'])
    store = [x if isinstance(x, float) else np.nan for x in store]
      # Convert output into pandas data frame
    df = pd.DataFrame({'Week': [i + 1 for i in range(len(date_store))], 'NDSI': store})
    return df

In [51]:
thumbnail_url = image.getThumbUrl({
    'bands': 'NDSI',
    'min': 0,
    'max': 3000,
    'region': region
})
IPython.display.HTML('Thumbnail URL: <a href = {0}>{0} </a>'.format(thumbnail_url))

In [52]:
IPython.display.Image(url = thumbnail_url)

In [0]:
filtered = images.filterDate('2014-01-01', '2014-12-31').median()

In [0]:
def GetTileLayerURl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}},{{y}}?token={token}"
  return tile_url_template.format(**map_id)

def ReplaceOverlayLayers(map_object, ee_image_object):
  for layer in map_object.layers[1:]:
    map_object.remove_layer(layer)
  tile_url = GetTileLayerURl(ee_image_object)
  map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

map2 = ipyleaflet.Map(zoom=3, layout = {'height': '400px'})
ReplaceOverlayLayers(map2, filtered)

In [48]:
display(map2)

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…